##### 导入库

In [57]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import tree
from collections import Counter
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SVMSMOTE,SMOTE
from sklearn.model_selection import GridSearchCV
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
import numpy as np
# from joblib import load,
import joblib
import onnxruntime as rt

from sklearn2pmml.pipeline import PMMLPipeline

from sklearn2pmml import sklearn2pmml
from pypmml import Model

##### 选择数据

In [58]:
n='n.csv'

p={
    "all":'all.csv',
    "cns":'cns.csv',
    "opi":'opi.csv',
    "other":'other.csv'
}

way="opi"


##### 读取数据

In [59]:
fea_dict={
    'all':[
"fft_imag_3",
"fft_angle_3",
"alt_stderr_min_50",
"alt_rvalue_min_50",
"fft_real_2",
"fft_real_5",
"erbc_num10_focus2",
"fft_imag_6",
"lumpiness"
],


 'opi':[
"fft_imag_3",
"erbc_num10_focus4",
"alt_rvalue_min_50",
"fft_angle_3",
"erbc_num10_focus2",
"fft_real_5"
],

 'cns':[
"fft_imag_3",
"fft_angle_3",
"fft_real_5",
"minindex",
"lumpiness",
"fft_imag_6",
"alt_rvalue_min_50",
"fft_angle_2",
"erbc_num10_focus2",
"S_down",
"retindex",
"erbc_num10_focus4",
"mac",
"alt_stderr_min_50",
],

'other':[
  "fft_imag_3",
  "fft_angle_3",
  "alt_stderr_min_50",
  "fft_imag_6",
  "fft_real_2",
  "alt_rvalue_min_50",
  "fft_real_5",
  "minvalue",
  "V2",
  "F2",
  "retindex"
]
}

fea_list=fea_dict[way]

def read_data(tit):
    path = 'E:\\desktop\\data\\app\\223\\fea\\ok\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data

n_fea=read_data(n)
p_fea=read_data(p[way])

n_fea["tag"]=[0 for i in range(len(n_fea))]
p_fea["tag"]=[1 for i in range(len(p_fea))]

print(len(n_fea),len(p_fea))

count_fea=[]
for key in fea_dict.keys():
    count_fea=count_fea+fea_dict[key]

count_fea=set(count_fea)

521 176


##### 过采样

In [60]:
#过采样
over=SMOTE(random_state=0)
# over=SVMSMOTE(random_state=0)
np_fea=shuffle(pd.concat([n_fea,p_fea],axis=0))
x_data_=np_fea[fea_list]
y_data_=np_fea["tag"]

x_data,y_data=over.fit_resample(x_data_,y_data_)
x_data=x_data.astype('float32')
#切分
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3)

print(x_data.shape)

(1042, 6)


In [61]:
def jugde(y_pred,y_true):
    tn,fn,tp,fp=0,0,0,0
    for i in range(len(y_pred)):
        
        if y_pred[i]==y_true[i]:
            if y_true[i]:
                tp+=1
            else:
                tn+=1
        else:
            if y_true[i]:
                fp+=1
            else:
                fn+=1

    print("检出率： ",tp,"/",tp+fp,' ',round(tp*100/(tp+fp),2),"%")
    print("误判率： ",fn,"/",tn+fn,' ',round(fn*100/(tn+fn),2),"%")
    print("精确率： ",(tn+tp),"/",(tn+fp+tp+fp),' ',round((tn+tp)*100/(tn+fp+tp+fp),2),"%")


##### 搭建模型

In [62]:
# etc=RandomForestClassifier(random_state=0)
# etc.fit(x_train,y_train)
# print(x_train.dtypes)


etc = PMMLPipeline([
	("classifier", RandomForestClassifier(random_state=0))
])
etc.fit(x_train, y_train)




print("测试集中：")
y_pred=etc.predict(x_test)
jugde(y_pred,list(y_test))

print("所有数据中：")
y_pred=etc.predict(x_data_)
jugde(y_pred,list(y_data_))

etc.score(x_test,y_test)

测试集中：
检出率：  133 / 142   93.66 %
误判率：  19 / 171   11.11 %
精确率：  285 / 303   94.06 %
所有数据中：
检出率：  172 / 176   97.73 %
误判率：  19 / 521   3.65 %
精确率：  674 / 682   98.83 %


0.9105431309904153

##### 转换保存

In [63]:
sklearn2pmml(etc, way+".pmml", with_repr = True)

print(way+"保存成功")

model = Model.load(way+".pmml")

print("测试集中：")
y_pred=model.predict(x_test)
print(len(y_pred[y_pred['probability(0)']>0.5])/len(x_test))

# jugde(y_pred,list(y_test))

print("所有数据中：")
y_pred=model.predict(x_data_)
# jugde(y_pred,list(y_data_))


Exception in thread Thread-22:
Traceback (most recent call last):
  File "d:\anaconada\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "d:\anaconada\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "d:\anaconada\lib\subprocess.py", line 1479, in _readerthread
    buffer.append(fh.read())
  File "d:\anaconada\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb6 in position 0: invalid start byte


opi保存成功
测试集中：
0.5143769968051118
所有数据中：
